# 数据透视表与交叉表

数据透视表是数据分析的一个常用工具，是一个非常强大的汇总运算函数。在 Pandas 中使用`pivot_table()`来实现数据透视表，Pandas 还提供`cross_table()`函数实现数据交叉表，即一种特殊的数据透视表。本节将介绍它们的使用说明以及应用。

In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 使用`pivot_table()`创建透视表

在前面章节，介绍过使用`df.pivot()`来实现从长格式到宽格式的数据透视。在使用`df.pivot()`进行数据透视时，会指定新的行与列索引，当行列索引存在重复条目时会抛出异常。例如对于下面饭店顾客与流水数据：

In [2]:
%%writefile restaurant_customers.csv
日期,门店,餐时,顾客,流水
2018-01-01,Xuhui,午餐,120,2489.5
2018-01-01,Xuhui,夜市,300,27028.9
2018-01-01,Sheshan,午餐,200,3029.4
2018-01-01,Sheshan,夜市,300,15293.0
2018-01-02,Xuhui,早点,102,878.0
2018-01-02,Xuhui,午餐,122,2689.5
2018-01-02,Xuhui,夜市,303,27828.9
2018-01-02,Sheshan,早点,32,180.0
2018-01-02,Sheshan,午餐,210,3629.4
2018-01-02,Sheshan,夜市,310,15393.0
2018-01-02,Jingan,早点,42,322.0
2018-01-02,Jingan,午餐,252,5849.2
2018-01-02,Jingan,夜市,202,20492.0

Overwriting restaurant_customers.csv


使用`read_csv()`读取数据文件：

In [3]:
df = pd.read_csv('restaurant_customers.csv', parse_dates=['日期'])
df.head()

,日期,门店,餐时,顾客,流水
0,2018-01-01,Xuhui,午餐,120,2489.5
1,2018-01-01,Xuhui,夜市,300,27028.9
2,2018-01-01,Sheshan,午餐,200,3029.4
3,2018-01-01,Sheshan,夜市,300,15293.0
4,2018-01-02,Xuhui,早点,102,878.0


In [4]:
# 下面语句会抛出`ValueError`异常: 重复条目
# df.pivot(index='门店', columns='餐时', values=['顾客', '流水'])

可以使用函数`pd.pivot_table()`函数或`DataFrame`对象的`df.pivot_table()`方法来创建数据透视表。与`pivot()`的区别是，数据透视表会对重复条目进行聚合操作。使用语法为：
```python
pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
df.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
```
主要参数
- `index=None`, 字符串或列表，指定新对象的行标签索引
    - 指定原对象中的列或多个列
    - 使用列表或者嵌套列表
- `columns=None`, 字符串或列表，指定新对象列标签索引。
    - 指定原对象中的列或多个列
    - 使用列表或嵌套列表
- `values=None`，指定要聚合的数据字段名。默认使用全部数据列，与`columns`指定列生成层级索引。
- `aggfunc=mean`，`values`值的汇总函数，默认为mean,始终做聚合操作
    - 使用内置聚合函数(如`max`、`min`)
    - 使用自定义函数
    - 使用函数列表
    - 使用字典，键值为列名，值为函数
- `fill_value=None`，填充值。
- `margins=False`, 布尔数。是否显示汇总数据。
- `margins_name='All'`, 汇总数据列名。
- `dropna=True`，布尔数。是否删除全为NAN值的列。
- `squeeze=False`，布尔数。是否在可行情况下对返回数据进行降维。
- `observed=False`，布尔数。针对类别数据。

下面来创建饭店-流水的数据透视表，新对象的行索引使用“门店”列，列索引使用“餐时”列，数据指定为“顾客，流水”列，聚合函数使用缺省值，即求均值：

In [5]:
df.pivot_table(index='门店', columns='餐时', values=['顾客', '流水'])

流水                     顾客              
餐时           午餐       夜市     早点     午餐     夜市     早点
门店                                                  
Jingan   5849.2  20492.0  322.0  252.0  202.0   42.0
Sheshan  3329.4  15343.0  180.0  205.0  305.0   32.0
Xuhui    2589.5  27428.9  878.0  121.0  301.5  102.0

如果指定`margins=True`，会显示汇总数据，缺省汇总数据列名为`All`，可以使用`margins_name`参数来指定：

In [6]:
df.pivot_table(index='门店', columns='餐时', values=['顾客', '流水'], margins=True, margins_name='总和')

流水                                    顾客                     \
餐时           午餐        夜市     早点            总和     午餐     夜市          早点   
门店                                                                         
Jingan   5849.2  20492.00  322.0   8887.733333  252.0  202.0   42.000000   
Sheshan  3329.4  15343.00  180.0   7504.960000  205.0  305.0   32.000000   
Xuhui    2589.5  27428.90  878.0  12182.960000  121.0  301.5  102.000000   
总和       3537.4  21207.16  460.0   9623.292308  180.8  283.0   58.666667   

                     
餐时               总和  
门店                   
Jingan   165.333333  
Sheshan  210.400000  
Xuhui    189.400000  
总和       191.923077

可以指定多个聚合函数：

In [7]:
df.pivot_table(index='门店', columns='餐时', values=['顾客', '流水'], aggfunc=['max', np.count_nonzero])

max                                count_nonzero                  \
             流水                   顾客                      流水           顾客      
餐时           午餐       夜市     早点   午餐   夜市   早点            午餐   夜市   早点 午餐 夜市   
门店                                                                             
Jingan   5849.2  20492.0  322.0  252  202   42           1.0  1.0  1.0  1  1   
Sheshan  3629.4  15393.0  180.0  210  310   32           2.0  2.0  1.0  2  2   
Xuhui    2689.5  27828.9  878.0  122  303  102           2.0  2.0  1.0  2  2   

            
            
餐时      早点  
门店          
Jingan   1  
Sheshan  1  
Xuhui    1

也可以为不同列指定不同聚合函数：

In [8]:
df.pivot_table(index='门店', columns='餐时', values=['顾客', '流水'], 
               aggfunc={'顾客': 'max', '流水': ['min', 'max']})

流水                                           顾客          
            max                     min                  max          
餐时           午餐       夜市     早点      午餐       夜市     早点   午餐   夜市   早点
门店                                                                    
Jingan   5849.2  20492.0  322.0  5849.2  20492.0  322.0  252  202   42
Sheshan  3629.4  15393.0  180.0  3029.4  15293.0  180.0  210  310   32
Xuhui    2689.5  27828.9  878.0  2489.5  27028.9  878.0  122  303  102

## 使用`crosstab()`创建交叉表

交叉表是一种特殊的透视表，用于计算分组频次。使用`pd.crosstab()`函数来创建交叉表，其使用语法为：
```python
pd.crosstab(data, index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, margins_name='All', dropna=True, normalize=False)
```
主要参数
- `index`, 字符串或列表，指定行索引。
- `columns`, 字符串或列表，指定列索引。
- `values=None`，数组，表示聚合数据。
- `rownames=None`，行分组键。
- `colnames=None`，列分组键。
- `aggfunc=mean`，聚合函数。
- `margins=False`, 布尔数。是否显示汇总数据。
- `dropna=True`，布尔数。是否删除全为NAN值的列。
- `normalize=False`，是否对值进行标准化。

In [9]:
pd.crosstab(df['门店'], df['餐时'])

餐时,午餐,夜市,早点
门店,,,
Jingan,1,1,1
Sheshan,2,2,1
Xuhui,2,2,1
